In [6]:
id2tag = pd.read_csv('data/カテゴリー・タグ定義.csv', encoding='cp932')['タグ'].to_dict()
tag2id = {v:k for k, v in id2tag.items()}

In [9]:
import pandas as pd

# データの読み込み
filepath = "data/20230926_ft.csv"
df = pd.read_csv(filepath)[['comment', 'tag']]
display(df['tag'].value_counts())

# ニュートラルを除外
# df = df[df['sentiment'] != 'NEUTRAL']

# ラベルを数値に変換
df['tag'] = df['tag'].map(tag2id)
display(df['tag'].value_counts())

df.rename(columns={'comment':'text', 'tag':'label'}, inplace=True)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], train_size=0.5, random_state=42)

X_test, X_eval, y_test, y_eval = train_test_split(
    X_test, y_test, train_size=0.5, random_state=42)

tag
接客態度           319
レジ処理の速度        132
デイリー           123
商品在庫            62
店内レイアウト         56
商品陳列            51
商品知識            37
割引サービス          29
フード＆ドリンク        23
ホームグッズ          23
駐車場             23
店内の清潔度          21
デイリーグッズ         17
ヘルスケア           17
欠品              15
商品価格            15
ベジタブル・フルーツ      11
ペット＆ガーデン         9
ファッション＆カバン       7
トイレ設備            7
デリカ              7
その他サービス          7
リカー＆ワイン          7
菓子・珍味            6
ホームエレクトロニクス      5
サイクル             5
トイ＆バラエティ         5
その他設備            4
ハンディキャップ対応       4
安全対策             3
カーライフ            3
スマホパーツ           3
ホームインテリア         3
音響設備             2
フレッシュミート         2
スキンケア            2
ブランドファッション       1
メンズインナー          1
理美容家電            1
空調               1
照明設備             1
シューズ             1
寝具               1
コスメ              1
Name: count, dtype: int64

tag
40    319
41    132
30    123
45     62
43     56
44     51
39     37
46     29
29     23
8      23
52     23
42     21
15     17
16     17
48     15
38     15
36     11
17      9
19      7
55      7
34      7
49      7
31      7
33      6
2       5
10      5
1       5
57      4
56      4
54      3
0       3
3       3
7       3
51      2
35      2
28      2
20      1
18      1
6       1
53      1
50      1
22      1
11      1
25      1
Name: count, dtype: int64

In [10]:
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})
eval_df = pd.DataFrame({'text': X_eval, 'label': y_eval})

In [11]:
train_df.shape, test_df.shape, eval_df.shape

((536, 2), (268, 2), (269, 2))

In [15]:
import os

In [16]:
DATASET_DIR = 'data/fine-tuning'
os.makedirs(DATASET_DIR, exist_ok=True)

In [17]:
train_df.to_csv(f'./{DATASET_DIR}/train.csv', index=False, header=False)
test_df.to_csv(f'./{DATASET_DIR}/test.csv', index=False, header=False)
eval_df.to_csv(f'./{DATASET_DIR}/eval.csv', index=False, header=False)

In [21]:
# https://huggingface.co/docs/datasets/create_dataset
from datasets import DatasetBuilder, GeneratorBasedBuilder
import datasets
import csv

DATASET_DIR = 'data/fine-tuning'

class FTDataset(GeneratorBasedBuilder):
    def _info(self):
        return datasets.DatasetInfo(
            features=datasets.Features({
                'text': datasets.Value('string'),
                'label': datasets.ClassLabel(names=list(id2tag.values())),
            }),
        )
    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""

        train_path = f'./{DATASET_DIR}/train.csv'
        test_path = f'./{DATASET_DIR}/test.csv'
        eval_path = f'./{DATASET_DIR}/eval.csv'

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": train_path}),
            datasets.SplitGenerator(name=datasets.Split.TEST, gen_kwargs={"filepath": test_path}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": eval_path}),
        ]

    def _generate_examples(self, filepath):
        # CSVファイルを行ごとに読み込み、それぞれの行をHugging Faceデータセットの形式に変換
        with open(filepath, encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file)
            for id_, row in enumerate(csv_reader):
                yield id_, {
                    'text': row[0], 
                    'label': row[1], 
                }

In [22]:
from datasets import load_dataset

dataset = load_dataset('dataset_loader.py', name='category-tagging')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 536
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 268
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 269
    })
})

In [32]:
id2tag

{0: 'カーライフ',
 1: 'トイ＆バラエティ',
 2: 'ホームエレクトロニクス',
 3: 'スマホパーツ',
 4: 'ラブグッズ',
 5: 'パーティーグッズ',
 6: '理美容家電',
 7: 'ホームインテリア',
 8: 'ホームグッズ',
 9: 'カーペンター＆ツール',
 10: 'サイクル',
 11: '寝具',
 12: 'プロテイン・トレーニング',
 13: 'アウトドア',
 14: 'フューチャープロダクト',
 15: 'デイリーグッズ',
 16: 'ヘルスケア',
 17: 'ペット＆ガーデン',
 18: 'メンズインナー',
 19: 'ファッション＆カバン',
 20: 'ブランドファッション',
 21: 'インポート',
 22: 'シューズ',
 23: 'レディス・キッズインナー',
 24: 'アロマ',
 25: 'コスメ',
 26: 'ステーショナリー',
 27: 'レディスファッション雑貨',
 28: 'スキンケア',
 29: 'フード＆ドリンク',
 30: 'デイリー',
 31: 'リカー＆ワイン',
 32: 'ギフト',
 33: '菓子・珍味',
 34: 'デリカ',
 35: 'フレッシュミート',
 36: 'ベジタブル・フルーツ',
 37: 'フレッシュフィッシュ',
 38: '商品価格',
 39: '商品知識',
 40: '接客態度',
 41: 'レジ処理の速度',
 42: '店内の清潔度',
 43: '店内レイアウト',
 44: '商品陳列',
 45: '商品在庫',
 46: '割引サービス',
 47: '会計',
 48: '欠品',
 49: 'その他サービス',
 50: '照明設備',
 51: '音響設備',
 52: '駐車場',
 53: '空調',
 54: '安全対策',
 55: 'トイレ設備',
 56: 'ハンディキャップ対応',
 57: 'その他設備'}